In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import mne
from mne.datasets import eegbci
from mne import Epochs, pick_types, find_events
import os
import scipy.io as sio

In [3]:
def read_dataset(dataset,num):
    path = "dataset"+str(dataset)+"/"
    if dataset == 1:
        path += "A"
    elif dataset == 2:
        path += "S"
    elif dataset == 4:
        path += "B"
    e = []
    t = []
    for i in range(1,num+1):
        e.append(np.asarray(sio.loadmat(path+str(i).zfill(2)+"E.mat")['data']))
        t.append(np.asarray(sio.loadmat(path+str(i).zfill(2)+"T.mat")['data']))
    return t, e

In [4]:
e, t = read_dataset(4,9)

In [124]:
def prep_d1(data):
    dataset = []
    for i in range(len(data[0])):
        dataset.append(data[0][i])
    return np.array(dataset)

def prep_t4(data):
    data1 = list(data.tolist()[0][0][0][0])
    data2 = list(data.tolist()[0][1][0][0])
    data3 = list(data.tolist()[0][2][0][0])
    return np.array([data1, data2, data3])

def prep_e4(data):
    data1 = list(data.tolist()[0][0][0][0])
    data2 = list(data.tolist()[0][1][0][0])
    return np.array([data1, data2])


In [204]:
def features1(data):
    x = data[0][0][0][:,:22]
    trial_raw = data[0][0][1]
    trial = [num for e in trial_raw for num in e]
    y = [num for e in data[0][0][2] for num in e]
    atf = [num for e in data[0][0][5] for num in e]
    rate = data[0][0][3].tolist()
    
    return (x,trial,y,atf,rate[0][0])

def features4(data):
    x = data[0][:,:3]
    trial_raw = data[1]
    trial = [num for e in trial_raw for num in e]
    y = [num for e in data[2] for num in e]
    atf = [num for e in data[5] for num in e]
    rate = data[3].tolist()
    
    return (x,trial,y,atf,rate[0][0])

In [ ]:
def split_data(data):
    x, trial, y, atf, rate = data
    group1 = list()
    group2 = list()
    group3 = list()
    group4 = list()
    a = list(range(0, x.shape[0])) 
    
    z = list(zip(atf, y, trial))
    filtered = [(b,c) for a,b,c in z if a == 0]
    label, trial_filter = zip(*filtered)

    for i in a:
        if i in trial_filter:
            group1.append(x[i:i+2*rate])
            group2.append(x[i+int(2.5*rate):i+int(4.5*rate)])
            group3.append(x[i+4*rate:i+6*rate])
            group4.append(x[i+int(6.5*rate):i+int(8.5*rate)])
            
    arr1 = np.array(group1)
    arr2 = np.array(group2)
    arr3 = np.array(group3)
    arr4 = np.array(group4)
    label = np.array(label)
    
    return arr1, arr2, arr3, arr4, label

In [206]:
t1, e1 = read_dataset(1,9)
t4, e4 = read_dataset(4,9)

In [207]:
x_t4 = [features4(data2) for data in data_t4 for data2 in data]
x_e4 = [features4(data2) for data in data_e4 for data2 in data]
x_t1 = []
for i in range(len(data_t1)):
    if i == 3:
        for j in range(1,len(data_t1[i])):
            x_t1.append(features1(data_t1[i][j]))
    else:
        for j in range(3,len(data_t1[i])):
            x_t1.append(features1(data_t1[i][j]))
            
x_e1 = []
for i in range(len(data_e1)):
    for j in range(3,len(data_e1[i])):
        x_e1.append(features1(data_e1[i][j]))

In [208]:
print('t1','*'*50)
for x in x_t1:
    print(len(x[0]),len(x[2]),x[4])
print('e1','*'*50)
for x in x_e1:
    print(len(x[0]),len(x[2]),x[4])
print('t4','*'*50)
for x in x_t4:
    print(len(x[0]),len(x[2]),x[4])
print('e4','*'*50)
for x in x_e4:
    print(len(x[0]),len(x[2]),x[4])

t1 **************************************************
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
e1 **************************************************
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48 250
96735 48

In [209]:
len(x_t1)

54

In [210]:
len(x_e1)

54

In [211]:
len(x_t4)

27

In [212]:
len(x_e4)

18

In [ ]:
x1t = [split_data(x) for x in x_t1]
x1e = [split_data(x) for x in x_e1]
x4t = [split_data(x) for x in x_t4]
x4e = [split_data(x) for x in x_e4]